## IBM Profesional Data Science Certificate
### Coursera Capstone Project
#### Yefry Lopez

 This notebook will be mainly used for the capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
pwd

'C:\\Users\\DELL USER\\GithubProjects\\IBMDataScienceProfessionalCertificate\\Coursera_Capstone'

In [4]:
data = 'Toronto_data.xlsx'
toronto = pd.read_excel(data)

In [5]:
toronto.head(2)

,Postal Code,Borough,Neighborhood
0,M1S,Scarborough,Agincourt
1,M8W,Etobicoke,"Alderwood, Long Branch"


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
torontoA = toronto[toronto['Borough'] != 'Not assigned'].reset_index(drop=True)


In [9]:
torontoA.head()


,Postal Code,Borough,Neighborhood
0,M1S,Scarborough,Agincourt
1,M8W,Etobicoke,"Alderwood, Long Branch"
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North"
3,M2K,North York,Bayview Village
4,M5M,North York,"Bedford Park, Lawrence Manor East"


### Group multiple neighborhoods having same postal code

In [10]:
toronto_group = torontoA.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
toronto_group['Neighborhood'] = toronto_group['Neighborhood'].str.replace('/', ',')
toronto_group.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Dataframe shape

In [11]:
toronto_group.shape

(103, 3)

### Adding Lattitude and Longitude to the dataframe from csv files

In [12]:
#Read csv file
data = 'Geospatial_Coordinates.csv'
geo_data = pd.read_csv(data)

In [13]:
#Merge two dataframes with a inner join on 'Postal Code' column
geo_toronto = pd.merge(toronto_group, geo_data , on='Postal Code')

In [14]:
geo_toronto.shape

(103, 5)

In [15]:
geo_toronto.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### k-means Clustering Analysis

#### Segmenting and Clustering Neighborhoods in Toronto City

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [16]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Quickly examine the resulting dataframe.

In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(geo_toronto['Borough'].unique()),
        geo_toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto City

In [18]:
address = 'Toronto , Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Create a map of New York with neighborhoods superimposed on top.

In [21]:
#create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(geo_toronto['Latitude'], geo_toronto['Longitude'], geo_toronto['Borough'], geo_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.


However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in 'Scarborough'. So let's slice the original dataframe and create a new dataframe of the 'Scarborough' data.

In [19]:
scarborough_data = geo_toronto[geo_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Let's get the geographical coordinates of Scarborough.

In [20]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7729744, -79.2576479.


As we did with all of Toronto City, let's visualizat Scarborough the neighborhoods in it.

In [21]:
# create map of Manhattan using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [22]:
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W20G0AZ0L3UKPMOQY0DFYENROKAXDBFEGRYRLNPPBRAYR5ML
CLIENT_SECRET:US32PLWNVEJENEYNFVP20CUEXNKZ4JYBPGBAD3XUAVDEARYM


#### Let's explore the first neighborhood in our dataframe.

In [23]:
scarborough_data.loc[0, 'Neighborhood']

'Malvern, Rouge'

#### Get the neighborhood's latitude and longitude values.

In [24]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Malvern, Rouge within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [25]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=W20G0AZ0L3UKPMOQY0DFYENROKAXDBFEGRYRLNPPBRAYR5ML&client_secret=US32PLWNVEJENEYNFVP20CUEXNKZ4JYBPGBAD3XUAVDEARYM&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

#### Send the GET request and examine the resutls

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fe68d934536486d2a7ae4c3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

#### Function that extracts the category of the venue

In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we are ready to clean the json and structure it into a pandas dataframe.

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


And how many venues were returned by Foursquare?

In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


### 2. Explore Neighborhoods in Malvern, Rouge

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


#### Let's check the size of the resulting dataframe

In [32]:
print(scarborough_venues.shape)
scarborough_venues.head()

(93, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


Neighborhood              0
Neighborhood Latitude     0
Neighborhood Longitude    0
Venue                     0
Venue Latitude            0
Venue Longitude           0
Venue Category            0
dtype: int64

#### Let's check how many venues were returned for each neighborhood

In [33]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


#### Let's find out how many unique categories can be curated from all the returned venues

In [34]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


### 3. Analyze Each Neighborhood

In [35]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Arts & Crafts Store,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### And let's examine the new dataframe size.

In [36]:
scarborough_onehot.shape

(93, 56)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Arts & Crafts Store,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.250000,0.000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000,0.000000,0.25,0.0,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.25,0.000000,0.000000,0.000000,0.25,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000,0.000000,0.25,0.0,0.000000,0.000000
2,Cedarbrae,0.0,0.00,0.111111,0.0,0.111111,0.111111,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.111111,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.111111,0.111111,0.00,0.000000,0.111111,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.111111,0.000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000,0.000000,0.00,0.0,0.111111,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.00,0.000000,0.0,0.000000,0.071429,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.071429,0.000000,0.00,0.071429,0.0,0.0,0.000000,0.142857,0.071429,0.071429,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.071429,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.000,0.0,0.0,0.000000,0.071429,0.00,0.000000,0.142857,0.142857,0.00,0.0,0.000,0.000,0.000000,0.00,0.0,0.071429,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.000,0.0,0.5,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000,0.000000,0.00,0.0,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.142857,0.0,0.0,0.00,0.000000,0.142857,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.285714,0.000,0.000000,0.000000,0.00,0.142857,0.000000,0.000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.00,0.142857,0.000000,0.000000,0.00,0.0,0.000,0.000,0.000000,0.00,0.0,0.000000,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.0,0.00,0.000000,0.0,0.200000,0.000000,0.000000,0.000000,0.000000,0.2,0.1,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.1,0.000000,0.100,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.1,0.000,0.0,0.0,0.000000,0.000000,0.10,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000,0.000000,0.00,0.1,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.0,0.00,0.000000,0.0,0.000000,0.125000,0.000000,0.125000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.125000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000

#### Let's confirm the new size

In [38]:
scarborough_grouped.shape

(16, 56)

#### Let's print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3                     Lounge  0.25
4        American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.11
1    Athletics & Sports  0.11
2       Thai Restaurant  0.11
3                Bakery  0.11
4                  Bank  0.11


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0  Fast Food Restaurant  0.14
1           Pizza Place  0.14
2              Pharmacy  0.14
3          Noodle House  0.07
4     Convenience Store  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                   venue  freq
0    American Restaurant   0.

#### Let's put that into a _pandas_ dataframe

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff, Cliffside West",Skating Rink,General Entertainment,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Convenience Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Fast Food Restaurant,Pizza Place,Pharmacy,Noodle House,Gas Station,Thai Restaurant,Fried Chicken Joint,Bank,Italian Restaurant,Chinese Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Home Service,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


### 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [42]:
#Run k-means to cluster the neighborhood into 5 clusters

# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 4, 1, 1, 1, 1, 1])

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [43]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how ='inner')

scarborough_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1,Moving Target,Bar,Home Service,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Bank,Medical Center,Restaurant,Rental Car Location,Intersection,Mexican Restaurant,Electronics Store,Breakfast Spot,Department Store,College Stadium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Home Service,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Convenience Store


In [44]:
scarborough_merged.isnull().sum()

Postal Code               0
Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

#### Finally, let's visualize the resulting clusters

In [45]:

#visualize the resulting clusters

#visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       


In [46]:
map_clusters

### 5. Examine Clusters

#### Cluster 1

In [48]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,0,Skating Rink,General Entertainment,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant


#### Cluster 2

In [51]:

scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,Scarborough,1,Moving Target,Bar,Home Service,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Scarborough,1,Bank,Medical Center,Restaurant,Rental Car Location,Intersection,Mexican Restaurant,Electronics Store,Breakfast Spot,Department Store,College Stadium
4,Scarborough,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Convenience Store
6,Scarborough,1,Discount Store,Department Store,Convenience Store,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,Scarborough,1,Bakery,Bus Line,Ice Cream Shop,Bus Station,Park,Metro Station,Intersection,Soccer Field,Fast Food Restaurant,Coffee Shop
10,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Light Rail Station,Brewery,Pet Store,Chinese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint
11,Scarborough,1,Middle Eastern Restaurant,Sandwich Place,Auto Garage,Bakery,Vietnamese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
12,Scarborough,1,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,1,Fast Food Restaurant,Pizza Place,Pharmacy,Noodle House,Gas Station,Thai Restaurant,Fried Chicken Joint,Bank,Italian Restaurant,Chinese Restaurant


#### Cluster 3

In [52]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2,Playground,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


#### Cluster 4

In [53]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Home Service,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Cluster 5

In [54]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,4,American Restaurant,Motel,Home Service,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
